# Pre-requisites  
- download dataset.zip,  
- unzip dataset,    
- mount drive,  

In [ ]:
# Package installations
from IPython.utils import io
with io.capture_output() as captured:
    !pip install gdown
    # !pip install torch==1.3.1+cu100 torchvision==0.4.2+cu100 -f https://download.pytorch.org/whl/torch_stable.html
    !pip install psutil
    !pip install neptune-client
    !pip install neptune-client neptune-tensorboard
    !pip install neptune-contrib
    !pip install imgaug==0.2.6
    !pip3 install --upgrade Pillow
    !pip install mxnet

In [ ]:
# Download VOC2012
# !gdown https://drive.google.com/uc?id=1PDTEuTnWJZNWogxYdqYGOlEZHK8dYET9

# Download custom Kvasir-v2
# ! gdown https://drive.google.com/uc?id=1WG5F7VVQe6mNupR1LLirkG_Apx6Bup5J

In [ ]:
# with io.capture_output() as captured:
  # !unzip -q VOC2012.zip
#   !unzip -q Kvasir-v2.zip
  # !rm -rf VOC2012.zip
#   !rm -rf Kvasir-v2.zip

In [ ]:
import neptune
from neptunecontrib.monitoring.keras import NeptuneMonitor
from IPython.utils import io
# connect run to project
NEPTUNE_TOKEN = 'eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJhMGUxMmQ1NC00ZDU4LTQ4ZGYtOWJjOC0xYTJkYjJmYmJiZDMifQ=='
run = neptune.init(project_qualified_name='ch.kalavritinos/OAA', api_token=NEPTUNE_TOKEN)

In [ ]:
from IPython.utils import io

In [ ]:
with io.capture_output() as captured:
  from google.colab import drive 
  drive.mount('/content/drive')
%cd /content/drive/MyDrive/MAI/thesis/source

/content/drive/MyDrive/MAI/thesis/source


In [ ]:
# %cd /content/drive/MyDrive/MAI/thesis/source/OAA/results_voc/
# !ls

In [ ]:
# !zip -r -q exp2.zip exp2

In [ ]:
# !gdown https://drive.google.com/drive/folders/uc?id=1P491c2_U7q6oYvgMR4bcVsoHVcsnGYA0
# !unzip -q SEAM_model.zip
# !rm -rf SEAM_model.zip

# OAA segmentation train
Imports

Change directory to OAA-PyTorch




In [ ]:
%cd OAA/

/content/drive/MyDrive/MAI/thesis/source/OAA


In [ ]:
from torchvision import transforms

In [ ]:
import sys
sys.path.append('scripts')
sys.path.append('utils')
sys.path.append('models')
import numpy as np
import torch
import random
from torch.utils.data import DataLoader
from torchvision import transforms
import voc12.data
from utils import pyutils, imutils, torchutils
import argparse
import importlib
import neptune
import os

torch.cuda.is_available()

False

In [ ]:
import PIL
PIL.__version__

'8.3.2'

# PARAMETERS

In [ ]:
root_dir = 'OAA'
data_root = '/content/drive/MyDrive/MAI/thesis/source/kvasir-dataset-v2'   # dataset directory
train_list = 'kvasirv2/train.txt'               # list of train images
test_list = 'kvasirv2/val.txt'                  # list of val images
snapshot_dir = 'checkpoints/train/exp5/'        # where to save models
att_dir = './results_kvasir/exp5/attention/'    # where to save attentions
dataset = 'kvasir'                              # dataset used


epoch = 14
lr = 0.001
batch_size = 18
input_size = 256
disp_interval = 100
num_classes = 8
num_workers = 2
weight_decay = 0.0005
decay_points = '5,10'

crop_size = 224         # for cropping images
threshold = 0.6         # for probabilities
disp_interval = 100     # display interval
resume  = False         # resume training 
global_counter = 0      
current_epoch = 0       # number of current epoch for resuming

In [ ]:
root_dir = 'OAA'
data_root = '/content/drive/MyDrive/MAI/thesis/source/VOCdevkit/VOC2012'      # dataset directory
# data_root = '/content/VOC2012'                                              # dataset directory

train_list = 'voc12/train_aug.txt'                # list of train images
test_list = 'voc12/val_oaa.txt'                   # list of val images
snapshot_dir = 'checkpoints/train_aff/exp2'       # where to save models
# att_dir = './results_voc/exp2/attention/'       # where to save attentions
dataset = 'voc'                                   # dataset used


epoch = 8
lr = 0.01
batch_size = 16
input_size = 256
disp_interval = 100
num_classes = 20
num_workers = 2
weight_decay = 0.0005
decay_points = '5,10'

session_name = "resnet38_aff"
crop_size = 224         # for cropping images
threshold = 0.6         # for probabilities
disp_interval = 100     # display interval
resume  = True          # resume training 
global_counter = 0      
current_epoch = 0       # number of current epoch for resuming

In [ ]:
weights = '/content/drive/MyDrive/MAI/thesis/source/pretrained_models/ilsvrc_cls.params'
# For Kvasir
la_crf_dir = 'results_kvasir/exp8/results_crf_4.0'
ha_crf_dir = 'results_kvasir/exp8/results_crf_24.0'

In [ ]:
# For VOC2012
la_crf_dir = 'results_voc/exp2/results_crf_4.0'
ha_crf_dir = 'results_voc/exp2/results_crf_24.0'

# data.py  
for Kvasir-v2

In [ ]:
# IMG_FOLDER_NAME = "polyps/"
# ANNOT_FOLDER_NAME = "polyps/masks"

# CAT_LIST = ['dyed-lifted-polyp',
#             'dyed-resection-margins',
#             'esophagitis',
#             'normal-cecum',
#             'normal-pylorus',
#             'normal-z-line', 
#             'polyps',
#             'ulcerative-colitis']

# CAT_NAME_TO_NUM = dict(zip(CAT_LIST,range(len(CAT_LIST))))


# def load_image_label_list_from_npy(img_name_list):
#     cls_labels_dict = np.load('kvasirv2/cls_labels.npy', allow_pickle=True).item()
#     # print(cls_labels_dict)
#     return [cls_labels_dict[img_name] for img_name in img_name_list]

# def get_img_path(img_name, dataset_root):
#     return os.path.join(dataset_root, img_name)

# def load_img_name_list(dataset_path):
#     img_gt_name_list = open(dataset_path).read().splitlines()
#     img_name_list = [img_gt_name.split(' ')[0][-40:-4] for img_gt_name in img_gt_name_list]
#     folder_paths_list = [img_gt_name.split(' ')[0] for img_gt_name in img_gt_name_list]
#     return img_name_list, folder_paths_list

# def load_label_list(dataset_path):
#     zero_array = np.zeros(8, dtype=np.float32)
#     list_of_arrays = []
#     for i in range(8):
#         z = np.zeros(8, dtype=np.float32)
#         z[i]=1.0
#         list_of_arrays.append(z)
#     #print(list_of_arrays)
#     img_name_list = open(dataset_path).read().splitlines()
#     label_list = [list_of_arrays[int(img_name[-1:])-1] for img_name in img_name_list]
#     return label_list

# class KvasirImageDataset(Dataset):
#     def __init__(self, img_name_list_path, dataset_root, transform=None):
#         self.img_name_list, self.folder_paths_list = load_img_name_list(img_name_list_path)
#         self.dataset_root = dataset_root
#         self.transform = transform

#     def __len__(self):
#         return len(self.img_name_list)

#     def __getitem__(self, idx):
#         name = self.img_name_list[idx]
#         path = self.folder_paths_list[idx]
#         #print(get_img_path(path, self.dataset_root))
#         img = PIL.Image.open(get_img_path(path, self.dataset_root)).convert("RGB")
#         # img = torch.from_numpy(np.array(img))

#         if self.transform:
#             img = self.transform(img)
#         return name, img

# class KvasirClsDataset(KvasirImageDataset):           # inherit init from previous class
#     def __init__(self, img_name_list_path, dataset_root, transform=None):
#         super().__init__(img_name_list_path, dataset_root, transform)
#         self.label_list = load_image_label_list_from_npy(self.img_name_list)        # get list from .npy file
#         #self.label_list = load_image_label_list_from_xml(self.img_name_list, self.dataset_root)

#     def __getitem__(self, idx):
#         name, img = super().__getitem__(idx)
#         label = torch.from_numpy(self.label_list[idx])
#         return name, img, label

# class KvasirClsDatasetMSF(KvasirClsDataset):
#     def __init__(self, img_name_list_path, data_root, scales, inter_transform=None, unit=1):
#         super().__init__(img_name_list_path, data_root, transform=None)
#         self.scales = scales
#         self.unit = unit
#         self.inter_transform = inter_transform

#     def __getitem__(self, idx):
#         name, img, label = super().__getitem__(idx)
#         rounded_size = (int(round(img.size[0]/self.unit)*self.unit), int(round(img.size[1]/self.unit)*self.unit))

#         ms_img_list = []
#         for s in self.scales:
#             target_size = (round(rounded_size[0]*s),
#                            round(rounded_size[1]*s))
#             s_img = img.resize(target_size, resample=PIL.Image.CUBIC)
#             ms_img_list.append(s_img)

#         if self.inter_transform:
#             for i in range(len(ms_img_list)):
#                 ms_img_list[i] = self.inter_transform(ms_img_list[i])

#         msf_img_list = []
#         for i in range(len(ms_img_list)):
#             msf_img_list.append(ms_img_list[i])
#             msf_img_list.append(np.flip(ms_img_list[i], -1).copy())
#         return name, msf_img_list, label

# class KvasirClsDatasetMS(KvasirClsDataset):
#     def __init__(self, img_name_list_path, dataset_root, scales, inter_transform=None, unit=1):
#         super().__init__(img_name_list_path, dataset_root, transform=None)
#         self.scales = scales
#         self.unit = unit
#         self.inter_transform = inter_transform

#     def __getitem__(self, idx):
#         name, img, label = super().__getitem__(idx)
#         rounded_size = (int(round(img.size[0]/self.unit)*self.unit), int(round(img.size[1]/self.unit)*self.unit))
#         ms_img_list = []
#         for s in self.scales:
#             target_size = (round(rounded_size[0]*s),
#                            round(rounded_size[1]*s))
#             s_img = img.resize(target_size, resample=PIL.Image.CUBIC)
#             ms_img_list.append(s_img)

#         if self.inter_transform:
#             for i in range(len(ms_img_list)):
#                 ms_img_list[i] = self.inter_transform(ms_img_list[i])

#         return name, ms_img_list, label

# Losses

In [ ]:
import torch.nn as nn
class DiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceLoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice = (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        return 1 - dice

class DiceBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(DiceBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        intersection = (inputs * targets).sum()
        dice_loss = 1 - (2.*intersection + smooth)/(inputs.sum() + targets.sum() + smooth)
        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        Dice_BCE = BCE + dice_loss
        return Dice_BCE

class IoULoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoULoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection
        IoU = (intersection + smooth)/(union + smooth)

        return -IoU

class IoUBCELoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(IoUBCELoss, self).__init__()

    def forward(self, inputs, targets, smooth=1):
        inputs = torch.sigmoid(inputs)
        #flatten label and prediction tensors
        inputs = inputs.view(-1)
        targets = targets.view(-1)
        #intersection is equivalent to True Positive count
        #union is the mutually inclusive area of all labels & predictions
        intersection = (inputs * targets).sum()
        total = (inputs + targets).sum()
        union = total - intersection
        IoU = - (intersection + smooth)/(union + smooth)

        BCE = F.binary_cross_entropy(inputs, targets, reduction='mean')
        IoU_BCE = BCE + IoU

        return IoU_BCE

# imutils.py


In [ ]:
import PIL.Image
import random
import numpy as np

"""Modified classes to include saliency maps as well
        Classes Included:
            RandomResizeLong()
            RandomCrop()
            AvgPool2d()
            RandomHorizontalFlip()
            CenterCrop()
            RescaleNearest()

        Functions included: 
            get_random_crop_box()
            crop_with_box()
            random_crop()
            HWC_to_CHW()
            crf_inference()
            bb_IOU()
            large_rect()

"""
class RandomResizeLong():
    """Resizes images from dataset using PIL to random shape between min_long and max_long"""

    def __init__(self, min_long, max_long):
        self.min_long = min_long
        self.max_long = max_long

    def __call__(self, img, sal=None):
        target_long = random.randint(self.min_long, self.max_long)
        w, h = img.size

        if w < h:
            target_shape = (int(round(w * target_long / h)), target_long)
        else:
            target_shape = (target_long, int(round(h * target_long / w)))

        img = img.resize(target_shape, resample=PIL.Image.CUBIC)
        if sal:
           sal = sal.resize(target_shape, resample=PIL.Image.CUBIC)
           return img, sal
        return img


class RandomCrop():
    """Randomly crops images"""
    def __init__(self, cropsize):
        self.cropsize = cropsize

    def __call__(self, imgarr, sal=None):
        print(np.shape(imgarr) )
        # h, w = np.shape(imgarr)                  # height, width
        w, h = img.size
        ch = min(self.cropsize, h)
        cw = min(self.cropsize, w)

        w_space = w - self.cropsize
        h_space = h - self.cropsize

        if w_space > 0:
            cont_left = 0
            img_left = random.randrange(w_space+1)
        else:
            cont_left = random.randrange(-w_space+1)
            img_left = 0

        if h_space > 0:
            cont_top = 0
            img_top = random.randrange(h_space+1)
        else:
            cont_top = random.randrange(-h_space+1)
            img_top = 0

        container = np.zeros((self.cropsize, self.cropsize, np.shape(imgarr)[-1]), np.float32)
        container[cont_top:cont_top+ch, cont_left:cont_left+cw] = \
            imgarr[img_top:img_top+ch, img_left:img_left+cw]
        if sal is not None:
            container_sal = np.zeros((self.cropsize, self.cropsize,1), np.float32)
            container_sal[cont_top:cont_top+ch, cont_left:cont_left+cw,0] = \
                sal[img_top:img_top+ch, img_left:img_left+cw]
            return container, container_sal

        return container

def get_random_crop_box(imgsize, cropsize):
    h, w = imgsize

    ch = min(cropsize, h)
    cw = min(cropsize, w)

    w_space = w - cropsize
    h_space = h - cropsize

    if w_space > 0:
        cont_left = 0
        img_left = random.randrange(w_space + 1)
    else:
        cont_left = random.randrange(-w_space + 1)
        img_left = 0

    if h_space > 0:
        cont_top = 0
        img_top = random.randrange(h_space + 1)
    else:
        cont_top = random.randrange(-h_space + 1)
        img_top = 0

    return cont_top, cont_top+ch, cont_left, cont_left+cw, img_top, img_top+ch, img_left, img_left+cw

def crop_with_box(img, box):
    if len(img.shape) == 3:
        img_cont = np.zeros((max(box[1]-box[0], box[4]-box[5]), max(box[3]-box[2], box[7]-box[6]), img.shape[-1]), dtype=img.dtype)
    else:
        img_cont = np.zeros((max(box[1] - box[0], box[4] - box[5]), max(box[3] - box[2], box[7] - box[6])), dtype=img.dtype)
    img_cont[box[0]:box[1], box[2]:box[3]] = img[box[4]:box[5], box[6]:box[7]]
    return img_cont


def random_crop(images, cropsize, fills):
    if isinstance(images[0], PIL.Image.Image):
        imgsize = images[0].size[::-1]
    else:
        imgsize = images[0].shape[:2]
    box = get_random_crop_box(imgsize, cropsize)

    new_images = []
    for img, f in zip(images, fills):

        if isinstance(img, PIL.Image.Image):
            img = img.crop((box[6], box[4], box[7], box[5]))
            cont = PIL.Image.new(img.mode, (cropsize, cropsize))
            cont.paste(img, (box[2], box[0]))
            new_images.append(cont)

        else:
            if len(img.shape) == 3:
                cont = np.ones((cropsize, cropsize, img.shape[2]), img.dtype)*f
            else:
                cont = np.ones((cropsize, cropsize), img.dtype)*f
            cont[box[0]:box[1], box[2]:box[3]] = img[box[4]:box[5], box[6]:box[7]]
            new_images.append(cont)

    return new_images


class AvgPool2d():
    """Average pooling on 2d"""
    def __init__(self, ksize):
        self.ksize = ksize

    def __call__(self, img):
        import skimage.measure

        return skimage.measure.block_reduce(img, (self.ksize, self.ksize, 1), np.mean)


class RandomHorizontalFlip():
    """Randomly mirror images"""
    def __init__(self):
        return

    def __call__(self, img, sal=None):
        if bool(random.getrandbits(1)):
            #img = img.transpose(PIL.Image.FLIP_LEFT_RIGHT)
            img = np.fliplr(img).copy()
            if sal:
                #sal = sal.transpose(PIL.Image.FLIP_LEFT_RIGHT)
                sal = np.fliplr(sal).copy()
                return img, sal 
            return img
        else:
            if sal:
                return img, sal
            return img


class CenterCrop():
    """Crop images centered"""
    def __init__(self, cropsize, default_value=0):
        self.cropsize = cropsize
        self.default_value = default_value

    def __call__(self, npimg):
        h, w = npimg.shape[:2]

        ch = min(self.cropsize, h)
        cw = min(self.cropsize, w)

        sh = h - self.cropsize
        sw = w - self.cropsize

        if sw > 0:
            cont_left = 0
            img_left = int(round(sw / 2))
        else:
            cont_left = int(round(-sw / 2))
            img_left = 0

        if sh > 0:
            cont_top = 0
            img_top = int(round(sh / 2))
        else:
            cont_top = int(round(-sh / 2))
            img_top = 0

        if len(npimg.shape) == 2:
            container = np.ones((self.cropsize, self.cropsize), npimg.dtype)*self.default_value
        else:
            container = np.ones((self.cropsize, self.cropsize, npimg.shape[2]), npimg.dtype)*self.default_value

        container[cont_top:cont_top+ch, cont_left:cont_left+cw] = \
            npimg[img_top:img_top+ch, img_left:img_left+cw]

        return container


def HWC_to_CHW(tensor, sal=False):
    if sal:
        tensor = np.expand_dims(tensor, axis=0)
    else:
        tensor = np.transpose(tensor, (2, 0, 1))
    return tensor


class RescaleNearest():
    """Resize image using nearest neighmor interpolation"""
    def __init__(self, scale):
        self.scale = scale

    def __call__(self, npimg):
        import cv2
        return cv2.resize(npimg, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)


def bb_IOU(boxA, boxB):
    """COmputes IOU based on bounding boxes (predicted and ground truth)"""
    boxA = [float(aa) for aa in boxA]
    boxB = [float(bb) for bb in boxB]

    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    
    if xA >= xB or yA >= yB:
        return 0, 0
    # compute the area of intersection rectangle
    interArea = (xB - xA + 1) * (yB - yA + 1)

    # compute the area of both the prediction and ground-truth rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    recall = interArea / float(boxAArea)
    return iou, recall

def large_rect(rect):
    """find largest rectangles"""
    large_area = 0
    target = 0
    for i in range(len(rect)):
        area = rect[i][2]*rect[i][3]
        if large_area < area:
            large_area = area
            target = i

    x = rect[target][0]
    y = rect[target][1]
    w = rect[target][2]
    h = rect[target][3]

    return x, y, w, h


def crf_inference(img, probs, t=10, scale_factor=1, labels=21):
    import pydensecrf.densecrf as dcrf
    from pydensecrf.utils import unary_from_softmax

    h, w = img.shape[:2]
    n_labels = labels

    d = dcrf.DenseCRF2D(w, h, n_labels)

    unary = unary_from_softmax(probs)
    unary = np.ascontiguousarray(unary)
    unary = unary.reshape((n_labels,-1))
    d.setUnaryEnergy(unary)
    d.addPairwiseGaussian(sxy=3/scale_factor, compat=3)
    d.addPairwiseBilateral(sxy=80/scale_factor, srgb=13, rgbim=np.copy(img), compat=10)
    Q = d.inference(t)

    return np.array(Q).reshape((n_labels, h, w))


# train.py

In [ ]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):     # save points during training
    """Function for saving checkpoints"""
    savepath = os.path.join(snapshot_dir, filename)
    torch.save(state, savepath)
    print(f"Model saved to {savepath}")
    

def load_ckp(checkpoint_fpath, model, optimizer):
    checkpoint = torch.load(checkpoint_fpath)
    from collections import OrderedDict
    new_state_dict = OrderedDict()
    for k, v in checkpoint['state_dict'].items():
        name = k[7:]                                  # remove `module.`
        new_state_dict[name] = v
    
    model.load_state_dict(new_state_dict)
    optimizer.load_state_dict(checkpoint['optimizer'])
    print('weights loaded!')
    return model, optimizer, checkpoint['epoch']+1, checkpoint['global_counter']

# Train

In [ ]:
torch.cuda.device_count()

0

In [ ]:
from utils.LoadData import train_data_loader
from torch.optim.lr_scheduler import MultiplicativeLR

def worker_init_fn(worker_id):
        np.random.seed(1 + worker_id)

def train(input_size = 224,
          disp_interval = 50,
          num_classes = 20,
          num_workers = 2,
          threshold = 0.6,          # for probabilities
          resume  = False,          # resume training 
          current_epoch = 0,
          batch_size = 16,
          global_counter = 0,
          momentum = 0.9,
          network="network.resnet38_aff"):

    if not os.path.exists(snapshot_dir):
        os.makedirs(snapshot_dir)

    model = getattr(importlib.import_module(network), 'Net')()
    current_epoch = current_epoch
    train_dataset = voc12.data.VOC12AffDataset(train_list, label_la_dir=la_crf_dir, label_ha_dir=ha_crf_dir,
                                               voc12_root=data_root, cropsize=crop_size, radius=5,
                                                joint_transform_list=[
                                                    None,
                                                    None,
                                                    imutils.RandomCrop(crop_size),
                                                    imutils.RandomHorizontalFlip()
                                                ],
                                                img_transform_list=[
                                                    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                                                    np.asarray,
                                                    model.normalize,
                                                    imutils.HWC_to_CHW
                                                ],
                                                label_transform_list=[
                                                    None,
                                                    None,
                                                    None,
                                                    imutils.AvgPool2d(8)
                                                ])
    train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                   pin_memory=True, drop_last=True, worker_init_fn=worker_init_fn)
    max_step = len(train_dataset) // batch_size * epoch
    print('max step: ', max_step)
    param_groups = model.get_parameter_groups()

    lambda1 = lambda epoch: 0.88#(1 - epoch* (len(train_dataset) // batch_size) / max_step) ** momentum
    optimizer = torch.optim.SGD([
        {'params': param_groups[0], 'lr': lr, 'weight_decay': weight_decay},
        {'params': param_groups[1], 'lr': 2*lr, 'weight_decay': 0},
        {'params': param_groups[2], 'lr': 10*lr, 'weight_decay': weight_decay},
        {'params': param_groups[3], 'lr': 20*lr, 'weight_decay': 0}
        ], lr=lr, weight_decay=weight_decay)
    

    scheduler = MultiplicativeLR(optimizer, lr_lambda=[lambda1, lambda1, lambda1, lambda1])

    PARAMS = {'dataset':dataset,
                        'network':network,
                        'epoch_nr': epoch,
                        'batch_size': batch_size,
                        'optimizer': 'SGD'}

    neptune.create_experiment(session_name, params=PARAMS)

    # Load weights
    if resume:
      model, optimizer, current_epoch = load_ckp(snapshot_dir+'/voc_epoch_1.pth', model, optimizer)
      # state_dict = torch.load(snapshot_dir+'/voc_epoch_1.pth')
      # global_counter = 1300 - 1
      # optimizer.global_step = global_counter - 1
      
    else:
      if weights[-7:] == '.params':
          import network.resnet38d
          weights_dict = network.resnet38d.convert_mxnet_to_torch(weights)
          model.load_state_dict(weights_dict, strict=False)
      else:
          weights_dict = torch.load(weights)
          model.load_state_dict(weights_dict, strict=False)
    
    model = torch.nn.DataParallel(model).cuda()
    model.to('cuda:0')
    # Set project and create run
    run = neptune.init(project_qualified_name='ch.kalavritinos/OAA', api_token=NEPTUNE_TOKEN)
    # print('Create class ...')
    
    model.train()

    avg_meter = pyutils.AverageMeter('loss', 'bg_loss', 'fg_loss', 'neg_loss', 'bg_cnt', 'fg_cnt', 'neg_cnt')
    timer = pyutils.Timer("Session started: ")


    while current_epoch < epoch:    
        for iter, pack in enumerate(train_data_loader):
            aff = model.forward(pack[0].to('cuda:0'))
            # aff.to('cuda:0')

            bg_label = pack[1][0].cuda(non_blocking=True)
            fg_label = pack[1][1].cuda(non_blocking=True)
            neg_label = pack[1][2].cuda(non_blocking=True)

            bg_count = torch.sum(bg_label) + 1e-5
            fg_count = torch.sum(fg_label) + 1e-5
            neg_count = torch.sum(neg_label) + 1e-5
            bg_count.to('cuda:0'), fg_count.to('cuda:0'), neg_count.to('cuda:0')

            bg_loss = torch.sum(- bg_label * torch.log(aff + 1e-5)) / bg_count
            fg_loss = torch.sum(- fg_label * torch.log(aff + 1e-5)) / fg_count
            neg_loss = torch.sum(- neg_label * torch.log(1. + 1e-5 - aff)) / neg_count
            bg_loss.to('cuda:0'), fg_loss.to('cuda:0'), neg_loss.to('cuda:0')

            loss = bg_loss/4 + fg_loss/4 + neg_loss/2
            loss.to('cuda:0')
            # print(loss)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            avg_meter.add({
                'loss': loss.item(),
                'bg_loss': bg_loss.item(), 'fg_loss': fg_loss.item(), 'neg_loss': neg_loss.item(),
                'bg_cnt': bg_count.item(), 'fg_cnt': fg_count.item(), 'neg_cnt': neg_count.item()
            })

            if (global_counter - 1) % disp_interval == 0:
                timer.update_progress(global_counter / max_step)

                print('Iter:%5d/%5d' % (global_counter-1, max_step),
                      'loss:%.4f %.4f %.4f %.4f' % avg_meter.get('loss', 'bg_loss', 'fg_loss', 'neg_loss'),
                      'cnt:%.0f %.0f %.0f' % avg_meter.get('bg_cnt', 'fg_cnt', 'neg_cnt'),
                      'imps:%.1f' % ((iter+1) * batch_size / timer.get_stage_elapsed()),
                      'lr: %.4f' % (optimizer.param_groups[0]['lr']), flush=True)

                # neptune.log_metric('loss', avg_meter.get('loss'))
                # neptune.log_metric('bg_loss', avg_meter.get('bg_loss'))
                # neptune.log_metric('fg_loss', avg_meter.get('fg_loss'))
                # neptune.log_metric('neg_loss', avg_meter.get('neg_loss'))
                avg_meter.pop()
            global_counter = global_counter + 1
            

        save_checkpoint(
                          {'epoch': current_epoch,
                            'state_dict': model.state_dict(),
                            'optimizer': optimizer.state_dict(),
                            'global_counter': global_counter
                          }, is_best=False,
                          filename='%s_epoch_%d.pth' %(dataset, current_epoch))
        # torch.save(model.module.state_dict(), session_name + current_epoch + '.pth')
        # print('model saved!...')
        current_epoch += 1
        scheduler.step()


In [ ]:
# train()

In [ ]:
model = getattr(importlib.import_module("network.resnet38_aff"), 'Net')()
model.state_dict().keys()

odict_keys(['conv1a.weight', 'b2.bn_branch2a.weight', 'b2.bn_branch2a.bias', 'b2.bn_branch2a.running_mean', 'b2.bn_branch2a.running_var', 'b2.bn_branch2a.num_batches_tracked', 'b2.conv_branch2a.weight', 'b2.bn_branch2b1.weight', 'b2.bn_branch2b1.bias', 'b2.bn_branch2b1.running_mean', 'b2.bn_branch2b1.running_var', 'b2.bn_branch2b1.num_batches_tracked', 'b2.conv_branch2b1.weight', 'b2.conv_branch1.weight', 'b2_1.bn_branch2a.weight', 'b2_1.bn_branch2a.bias', 'b2_1.bn_branch2a.running_mean', 'b2_1.bn_branch2a.running_var', 'b2_1.bn_branch2a.num_batches_tracked', 'b2_1.conv_branch2a.weight', 'b2_1.bn_branch2b1.weight', 'b2_1.bn_branch2b1.bias', 'b2_1.bn_branch2b1.running_mean', 'b2_1.bn_branch2b1.running_var', 'b2_1.bn_branch2b1.num_batches_tracked', 'b2_1.conv_branch2b1.weight', 'b2_2.bn_branch2a.weight', 'b2_2.bn_branch2a.bias', 'b2_2.bn_branch2a.running_mean', 'b2_2.bn_branch2a.running_var', 'b2_2.bn_branch2a.num_batches_tracked', 'b2_2.conv_branch2a.weight', 'b2_2.bn_branch2b1.weight',

In [ ]:
import network.resnet38_aff
model = network.resnet38_aff.Net()
model.state_dict().keys()

odict_keys(['conv1a.weight', 'b2.bn_branch2a.weight', 'b2.bn_branch2a.bias', 'b2.bn_branch2a.running_mean', 'b2.bn_branch2a.running_var', 'b2.bn_branch2a.num_batches_tracked', 'b2.conv_branch2a.weight', 'b2.bn_branch2b1.weight', 'b2.bn_branch2b1.bias', 'b2.bn_branch2b1.running_mean', 'b2.bn_branch2b1.running_var', 'b2.bn_branch2b1.num_batches_tracked', 'b2.conv_branch2b1.weight', 'b2.conv_branch1.weight', 'b2_1.bn_branch2a.weight', 'b2_1.bn_branch2a.bias', 'b2_1.bn_branch2a.running_mean', 'b2_1.bn_branch2a.running_var', 'b2_1.bn_branch2a.num_batches_tracked', 'b2_1.conv_branch2a.weight', 'b2_1.bn_branch2b1.weight', 'b2_1.bn_branch2b1.bias', 'b2_1.bn_branch2b1.running_mean', 'b2_1.bn_branch2b1.running_var', 'b2_1.bn_branch2b1.num_batches_tracked', 'b2_1.conv_branch2b1.weight', 'b2_2.bn_branch2a.weight', 'b2_2.bn_branch2a.bias', 'b2_2.bn_branch2a.running_mean', 'b2_2.bn_branch2a.running_var', 'b2_2.bn_branch2a.num_batches_tracked', 'b2_2.conv_branch2a.weight', 'b2_2.bn_branch2b1.weight',

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
  state_dict = torch.load(snapshot_dir+'/voc_epoch_7.pth')
else:
  state_dict = torch.load(snapshot_dir+'/voc_epoch_7.pth', map_location=torch.device('cpu'))
state_dict['optimizer']['state'].keys()
# new_state_dict.keys()

In [ ]:
param_groups = model.get_parameter_groups()
optimizer = torchutils.PolyOptimizer([
        {'params': param_groups[0], 'lr': lr, 'weight_decay': weight_decay},
        {'params': param_groups[1], 'lr': 2*lr, 'weight_decay': 0},
        {'params': param_groups[2], 'lr': 10*lr, 'weight_decay': weight_decay},
        {'params': param_groups[3], 'lr': 20*lr, 'weight_decay': 0}
        ], lr=lr, weight_decay=weight_decay, max_step=9915)

In [ ]:
from collections import OrderedDict

new_state_dict = OrderedDict()
for k, v in state_dict['state_dict'].items():
    name = k[7:] # remove `module.`
    new_state_dict[name] = v
# model.load_state_dict(new_state_dict)
# print('weights loaded!', current_epoch)

In [ ]:
from torch.utils.data import Dataset

IMG_FOLDER_NAME = "JPEGImages"
ANNOT_FOLDER_NAME = "Annotations"

CAT_LIST = ['aeroplane', 'bicycle', 'bird', 'boat',
        'bottle', 'bus', 'car', 'cat', 'chair',
        'cow', 'diningtable', 'dog', 'horse',
        'motorbike', 'person', 'pottedplant',
        'sheep', 'sofa', 'train',
        'tvmonitor']

CAT_NAME_TO_NUM = dict(zip(CAT_LIST,range(len(CAT_LIST))))

class VOC12ImageDataset(Dataset):
    def __init__(self, img_name_list_path, voc12_root, transform=None):
        # print('init of voc12 image dataset   ...   \n')
        self.img_name_list = load_img_name_list(img_name_list_path)
        # print('image name list len:', len(self.img_name_list))
        self.voc12_root = voc12_root
        self.transform = transform
        # print('------------')

    def __len__(self):
        # print('voc12 image dataset length 88888888')
        return len(self.img_name_list)

    def __getitem__(self, idx):
        # print('start of get item voc12 image dataset')
        name = self.img_name_list[idx]
        img = PIL.Image.open(get_img_path(name, self.voc12_root)).convert("RGB")
        # print('image opened!')

        if self.transform:
            img = self.transform(img)
        return name, img




class ExtractAffinityLabelInRadius():
    def __init__(self, cropsize, radius=5):
        # print('init of extract label')
        self.radius = radius
        self.search_dist = []

        for x in range(1, radius):
            self.search_dist.append((0, x))

        for y in range(1, radius):
            for x in range(-radius+1, radius):
                if x*x + y*y < radius*radius:
                    self.search_dist.append((y, x))

        self.radius_floor = radius-1

        self.crop_height = cropsize - self.radius_floor
        self.crop_width = cropsize - 2 * self.radius_floor
        # print('end of extract label')
        return

    def __call__(self, label):
        # print('start of _call_ in extract affinity ')
        labels_from = label[:-self.radius_floor, self.radius_floor:-self.radius_floor]
        labels_from = np.reshape(labels_from, [-1])

        labels_to_list = []
        valid_pair_list = []

        for dy, dx in self.search_dist:
            labels_to = label[dy:dy+self.crop_height, self.radius_floor+dx:self.radius_floor+dx+self.crop_width]
            labels_to = np.reshape(labels_to, [-1])

            valid_pair = np.logical_and(np.less(labels_to, 255), np.less(labels_from, 255))

            labels_to_list.append(labels_to)
            valid_pair_list.append(valid_pair)

        bc_labels_from = np.expand_dims(labels_from, 0)
        concat_labels_to = np.stack(labels_to_list)
        concat_valid_pair = np.stack(valid_pair_list)

        pos_affinity_label = np.equal(bc_labels_from, concat_labels_to)

        bg_pos_affinity_label = np.logical_and(pos_affinity_label, np.equal(bc_labels_from, 0)).astype(np.float32)

        fg_pos_affinity_label = np.logical_and(np.logical_and(pos_affinity_label, np.not_equal(bc_labels_from, 0)), concat_valid_pair).astype(np.float32)

        neg_affinity_label = np.logical_and(np.logical_not(pos_affinity_label), concat_valid_pair).astype(np.float32)
        # print('affinity labels before return extract affinity')
        return torch.from_numpy(bg_pos_affinity_label), torch.from_numpy(fg_pos_affinity_label), torch.from_numpy(neg_affinity_label)



def load_image_label_from_xml(img_name, voc12_root):
    from xml.dom import minidom             # to parse and modify xml docs

    el_list = minidom.parse(os.path.join(voc12_root, ANNOT_FOLDER_NAME,img_name + '.xml')).getElementsByTagName('name')

    multi_cls_lab = np.zeros((20), np.float32)      # init

    for el in el_list:
        cat_name = el.firstChild.data
        if cat_name in CAT_LIST:
            cat_num = CAT_NAME_TO_NUM[cat_name]
            multi_cls_lab[cat_num] = 1.0

    return multi_cls_lab            # return array with classes (has 0 or 1)

def load_image_label_list_from_xml(img_name_list, voc12_root):
    return [load_image_label_from_xml(img_name, voc12_root) for img_name in img_name_list]

def load_image_label_list_from_npy(img_name_list):
    # print('load image label list from npy')
    cls_labels_dict = np.load('voc12/cls_labels.npy', allow_pickle=True).item()

    return [cls_labels_dict[img_name] for img_name in img_name_list]

def get_img_path(img_name, voc12_root):
    # print('get img path')
    return os.path.join(voc12_root, IMG_FOLDER_NAME, img_name + '.jpg')



def load_img_name_list(dataset_path):
    img_gt_name_list = open(dataset_path).read().splitlines()
    img_name_list = [img_gt_name.split(' ')[0][-15:-4] for img_gt_name in img_gt_name_list]
    return img_name_list





class VOC12AffDataset(VOC12ImageDataset):
    def __init__(self, img_name_list_path, label_la_dir, label_ha_dir, cropsize, voc12_root, radius=5,
                 joint_transform_list=None, img_transform_list=None, label_transform_list=None):
        # print('voc12 Aff dataset init ... \n')
        super().__init__(img_name_list_path, voc12_root, transform=None)

        self.label_la_dir = label_la_dir
        self.label_ha_dir = label_ha_dir
        self.voc12_root = voc12_root

        self.joint_transform_list = joint_transform_list
        self.img_transform_list = img_transform_list
        self.label_transform_list = label_transform_list
        # print('before ExtractAffinityLabelInRadius')
        self.extract_aff_lab_func = ExtractAffinityLabelInRadius(cropsize=cropsize//8, radius=radius)
        # print('After ...')

    def __len__(self):
        # print('voc12 aff class _len_ : ', len(self.img_name_list))
        return len(self.img_name_list)

    def __getitem__(self, idx):
        name, img = super().__getitem__(idx)
        label_la_path = os.path.join(self.label_la_dir, name + '.npy')
        label_ha_path = os.path.join(self.label_ha_dir, name + '.npy')
        label_la = np.load(label_la_path, allow_pickle=True).item()
        label_ha = np.load(label_ha_path, allow_pickle=True).item()

        label = np.array(list(label_la.values()) + list(label_ha.values()))
        label = np.transpose(label, (1, 2, 0))
        # print('before joint TF')
        for joint_transform, img_transform, label_transform \
                in zip(self.joint_transform_list, self.img_transform_list, self.label_transform_list):

            if joint_transform:
                img_label = np.concatenate((img, label), axis=-1)
                img_label = joint_transform(img_label)
                img = img_label[..., :3]
                label = img_label[..., 3:]

            if img_transform:
                img = img_transform(img)
            if label_transform:
                label = label_transform(label)

        no_score_region = np.max(label, -1) < 1e-5
        label_la, label_ha = np.array_split(label, 2, axis=-1)
        label_la = np.argmax(label_la, axis=-1).astype(np.uint8)
        label_ha = np.argmax(label_ha, axis=-1).astype(np.uint8)
        label = label_la.copy()
        label[label_la == 0] = 255
        label[label_ha == 0] = 0
        label[no_score_region] = 255 # mostly outer of cropped region
        # print(type(label), label.shape, label.max(), label.min())     # 28x28 ndarray
        label = self.extract_aff_lab_func(label)
        # print(type(label))                  # tuple
        return img, label


        # label : 0=foreground, 1= bg, 2=negative


# Kvasir dataloaders


In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
test_dir = os.path.join('/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/test')
train_dir = os.path.join('/content/drive/My Drive/MAI/thesis/source/kvasir-dataset-v2-folds/1/train')

In [ ]:
def get_data_loader(data_dir, batch_size=32, train=True):
    # define how we augment the data for composing the batch-dataset in train and test step
    transform = {
        'train': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.RandomHorizontalFlip(),
            transforms.RandomVerticalFlip(),
            transforms.RandomRotation(90),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
        'test': transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(256),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
        ]),
    }

    # ImageFolder with root directory and defined transformation methods for batch as well as data augmentation
    if train:
      data = ImageFolderWithPaths(root=data_dir, transform=transform['train'])
    else:
      data = ImageFolderWithPaths(root=data_dir, transform=transform['test'])
    data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=True, num_workers=2)

    return data.class_to_idx, data_loader 

In [ ]:
torch.multiprocessing.freeze_support()
mapping, train_data_loader = get_data_loader(data_dir=train_dir, batch_size=32, train=True)
mapping, test_data_loader = get_data_loader(data_dir=test_dir, batch_size=32, train=False)

In [ ]:
import pathlib
train_data_dir = pathlib.Path(train_dir)
test_data_dir = pathlib.Path(test_dir)

In [ ]:
total_train = len(list(train_data_dir.glob('*/*.jpg')))
total_val = len(list(test_data_dir.glob('*/*.jpg')))
total_train, total_val

(7200, 800)

In [ ]:
#get the class names
CLASS_NAMES = np.array([item.name for item in train_data_dir.glob('*') if item.name != "LICENSE.txt"])
CLASS_NAMES

array(['normal-pylorus', 'polyps', 'normal-z-line', 'normal-cecum',
       'esophagitis', 'dyed-lifted-polyps', 'dyed-resection-margins',
       'ulcerative-colitis'], dtype='<U22')

In [ ]:
# Define parameters for training
dataset = 'kvasirv2'
batch_size = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(total_train/batch_size)
num_classes = len(CLASS_NAMES) # 8

epochs = 16
lr = 0.001
disp_interval = 100
weight_decay = 0.0005
decay_points = '5, 10'

In [ ]:
# from utils.LoadData import train_data_loader

def worker_init_fn(worker_id):
        np.random.seed(1 + worker_id)

input_size = 224
disp_interval = 50
num_classes = 20
num_workers = 2
threshold = 0.6
resume  = True
current_epoch = 0
batch_size = 16
global_counter = 0
network="network.resnet38_aff"
  
if not os.path.exists(snapshot_dir):
    os.makedirs(snapshot_dir)

model = getattr(importlib.import_module(network), 'Net')()
current_epoch = current_epoch
train_dataset = VOC12AffDataset(train_list, label_la_dir=la_crf_dir, label_ha_dir=ha_crf_dir,
                                            voc12_root=data_root, cropsize=crop_size, radius=5,
                                            joint_transform_list=[
                                                None,
                                                None,
                                                imutils.RandomCrop(crop_size),
                                                imutils.RandomHorizontalFlip()
                                            ],
                                            img_transform_list=[
                                                transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1),
                                                np.asarray,
                                                model.normalize,
                                                imutils.HWC_to_CHW
                                            ],
                                            label_transform_list=[
                                                None,
                                                None,
                                                None,
                                                imutils.AvgPool2d(8)
                                            ])
train_data_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers,
                                pin_memory=True, drop_last=True, worker_init_fn=worker_init_fn)
max_step = len(train_dataset) // batch_size * epoch
print('max step: ', max_step)

max step:  5288


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torchvision 

# functions to show an image

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

# get some random training images
dataiter = iter(train_data_loader)
images, labels, _ = dataiter.next()
imshow(torchvision.utils.make_grid(images))

In [ ]:
device

device(type='cpu')

In [ ]:
model.eval()
model.to(device)
for iter, pack in enumerate(train_data_loader):
      pack[0] = pack[0].to(device)
      pack[1][0] = pack[1][0].to(device)        # batch x 34 x 480
      pack[1][1] = pack[1][1].to(device)
      pack[1][2] = pack[1][2].to(device)

      aff = model.forward(pack[0].to(device))
      aff.to(device)

      bg_label = pack[1][0].cuda(non_blocking=True)
      fg_label = pack[1][1].cuda(non_blocking=True)
      neg_label = pack[1][2].cuda(non_blocking=True)

      bg_count = torch.sum(bg_label) + 1e-5
      fg_count = torch.sum(fg_label) + 1e-5
      neg_count = torch.sum(neg_label) + 1e-5
      bg_count.to(device), fg_count.to(device), neg_count.to(device)

      bg_loss = torch.sum(- bg_label * torch.log(aff + 1e-5)) / bg_count
      fg_loss = torch.sum(- fg_label * torch.log(aff + 1e-5)) / fg_count
      neg_loss = torch.sum(- neg_label * torch.log(1. + 1e-5 - aff)) / neg_count
      bg_loss.to(device), fg_loss.to(device), neg_loss.to(device)

      loss = bg_loss/4 + fg_loss/4 + neg_loss/2
      loss = loss.to(device)
      # print(loss)
      break



In [ ]:
bg_label.shape

torch.Size([16, 34, 480])

In [ ]:
bg_count, fg_count, neg_count

(tensor(113536., device='cuda:0'),
 tensor(34238., device='cuda:0'),
 tensor(5075., device='cuda:0'))

In [ ]:
img = pack[0].to(device)

In [ ]:
import matplotlib.pyplot as plt

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

In [ ]:
img[5].max(), img[5].min()

(tensor(2.6400, device='cuda:0'), tensor(-2.1179, device='cuda:0'))

In [ ]:
model.normalize

In [ ]:
for i in range(0,15):
  print(aff[i].max(), aff[i].min())

tensor(0.9990, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8896, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9955, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8800, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9991, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.9122, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9986, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.9008, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9987, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.9226, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9913, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8689, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9940, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8855, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9993, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8730, device='cuda:0', grad_fn=<MinBackward1>)
tensor(0.9975, device='cuda:0', grad_fn=<MaxBackward1>) tensor(0.8691, device='cuda:0', grad_fn=<MinBack

In [ ]:
imgs = dat[0]
labels = dat[1]

In [ ]:
labels[0].shape, labels[1].shape, labels[2].shape

(torch.Size([16, 34, 480]),
 torch.Size([16, 34, 480]),
 torch.Size([16, 34, 480]))

In [ ]:
labels[0][0]

tensor([[0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        ...,
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.],
        [0., 0., 0.,  ..., 1., 1., 1.]])

In [ ]:
param_groups = model.get_parameter_groups()
optimizer = torchutils.PolyOptimizer([
    {'params': param_groups[0], 'lr': lr, 'weight_decay': weight_decay},
    {'params': param_groups[1], 'lr': 2*lr, 'weight_decay': 0},
    {'params': param_groups[2], 'lr': 10*lr, 'weight_decay': weight_decay},
    {'params': param_groups[3], 'lr': 20*lr, 'weight_decay': 0}
    ], lr=lr, weight_decay=weight_decay, max_step=max_step)

PARAMS = {'dataset':dataset,
                    'network':network,
                    'epoch_nr': epoch,
                    'batch_size': batch_size,
                    'optimizer': 'PolyOptimizer'}

neptune.create_experiment(session_name, params=PARAMS)

# Load weights
if resume:
  model, optimizer, current_epoch = load_ckp(snapshot_dir+'/voc_epoch_7.pth', model, optimizer)
  # state_dict = torch.load(snapshot_dir+'/voc_epoch_1.pth')
  optimizer.global_step = global_counter
  
else:
  if weights[-7:] == '.params':
      import network.resnet38d
      weights_dict = network.resnet38d.convert_mxnet_to_torch(weights)
      model.load_state_dict(weights_dict, strict=False)
  else:
      weights_dict = torch.load(weights)
      model.load_state_dict(weights_dict, strict=False)

# model = torch.nn.DataParallel(model).cuda()
# model.cuda()
# Set project and create run
run = neptune.init(project_qualified_name='ch.kalavritinos/OAA', api_token=NEPTUNE_TOKEN)
# print('Create class ...')
model.train()

avg_meter = pyutils.AverageMeter('loss', 'bg_loss', 'fg_loss', 'neg_loss', 'bg_cnt', 'fg_cnt', 'neg_cnt')
timer = pyutils.Timer("Session started: ")

In [ ]:
while current_epoch < epoch:    
    for iter, pack in enumerate(train_data_loader):
        pack[0] = pack[0].to('cuda:0')
        pack[1][0] = pack[1][0].to('cuda:0')
        pack[1][1] = pack[1][1].to('cuda:0')
        pack[1][2] = pack[1][2].to('cuda:0')


        aff = model.forward(pack[0].to('cuda:0'))
        # aff.to('cuda:0')

        bg_label = pack[1][0].cuda(non_blocking=True)
        fg_label = pack[1][1].cuda(non_blocking=True)
        neg_label = pack[1][2].cuda(non_blocking=True)

        bg_count = torch.sum(bg_label) + 1e-5
        fg_count = torch.sum(fg_label) + 1e-5
        neg_count = torch.sum(neg_label) + 1e-5
        bg_count.to('cuda:0'), fg_count.to('cuda:0'), neg_count.to('cuda:0')

        bg_loss = torch.sum(- bg_label * torch.log(aff + 1e-5)) / bg_count
        fg_loss = torch.sum(- fg_label * torch.log(aff + 1e-5)) / fg_count
        neg_loss = torch.sum(- neg_label * torch.log(1. + 1e-5 - aff)) / neg_count
        bg_loss.to('cuda:0'), fg_loss.to('cuda:0'), neg_loss.to('cuda:0')

        loss = bg_loss/4 + fg_loss/4 + neg_loss/2
        loss = loss.to('cuda:0')
        # print(loss)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_meter.add({
            'loss': loss.item(),
            'bg_loss': bg_loss.item(), 'fg_loss': fg_loss.item(), 'neg_loss': neg_loss.item(),
            'bg_cnt': bg_count.item(), 'fg_cnt': fg_count.item(), 'neg_cnt': neg_count.item()
        })

        if (global_counter - 1) % disp_interval == 0:
            timer.update_progress(global_counter / max_step)

            print('Iter:%5d/%5d' % (global_counter-1, max_step),
                  'loss:%.4f %.4f %.4f %.4f' % avg_meter.get('loss', 'bg_loss', 'fg_loss', 'neg_loss'),
                  'cnt:%.0f %.0f %.0f' % avg_meter.get('bg_cnt', 'fg_cnt', 'neg_cnt'),
                  'imps:%.1f' % ((iter+1) * batch_size / timer.get_stage_elapsed()),
                  'lr: %.4f' % (optimizer.param_groups[0]['lr']), flush=True)

            neptune.log_metric('loss', avg_meter.get('loss'))
            neptune.log_metric('bg_loss', avg_meter.get('bg_loss'))
            neptune.log_metric('fg_loss', avg_meter.get('fg_loss'))
            neptune.log_metric('neg_loss', avg_meter.get('neg_loss'))
            avg_meter.pop()
        global_counter = global_counter + 1

    save_checkpoint(
                      {'epoch': current_epoch,
                        'state_dict': model.state_dict(),
                        'optimizer': optimizer.state_dict(),
                        'global_counter': global_counter
                      }, is_best=False,
                      filename='%s_epoch_%d.pth' %(dataset, current_epoch))
    # torch.save(model.module.state_dict(), session_name + current_epoch + '.pth')
    # print('model saved!...')
    current_epoch += 1


max step:  5288
https://app.neptune.ai/ch.kalavritinos/OAA/e/OAA-110


KeyboardInterrupt: ignored

In [ ]:
print(bg_loss.device, loss.device, fg_loss.device, neg_loss.device)
print(bg_count.device, loss.device, fg_count.device, neg_count.device)

cuda:0 cuda:0 cuda:0 cuda:0
cuda:0 cuda:0 cuda:0 cuda:0


In [ ]:
pack[0].device, pack[1][0].device

(device(type='cuda', index=0), device(type='cuda', index=0))

In [ ]:
pack[0].shape, pack[1][0].shape, pack[1][1].shape, pack[1][2].shape

(torch.Size([16, 3, 224, 224]),
 torch.Size([16, 34, 480]),
 torch.Size([16, 34, 480]),
 torch.Size([16, 34, 480]))

In [ ]:
optimizer.param_groups[0].keys()

dict_keys(['lr', 'weight_decay', 'momentum', 'dampening', 'nesterov', 'params'])

In [ ]:
model.device_ids

[0]

In [ ]:
train()

max step:  9915
https://app.neptune.ai/ch.kalavritinos/OAA/e/OAA-78
Session started:  Sun May 30 20:57:24 2021
Iter:    0/ 9915 loss:0.7188 0.9207 1.1738 0.3903 cnt:111303 36960 6617 imps:1.1 lr: 0.0100
Iter:   50/ 9915 loss:0.6518 0.6876 0.7764 0.5717 cnt:113898 40116 4858 imps:2.6 lr: 0.0100
Iter:  100/ 9915 loss:0.6302 0.5974 0.7157 0.6039 cnt:115074 37393 4328 imps:2.6 lr: 0.0099
Iter:  150/ 9915 loss:0.6248 0.5729 0.7065 0.6100 cnt:111791 41274 5411 imps:2.7 lr: 0.0099
Iter:  200/ 9915 loss:0.6208 0.5637 0.7118 0.6038 cnt:114947 38993 4774 imps:2.6 lr: 0.0098
Iter:  250/ 9915 loss:0.6029 0.5304 0.7124 0.5844 cnt:113523 40818 5037 imps:2.6 lr: 0.0098
Iter:  300/ 9915 loss:0.5919 0.5291 0.7022 0.5681 cnt:117885 38496 4648 imps:2.6 lr: 0.0097
Iter:  350/ 9915 loss:0.5495 0.5101 0.6559 0.5160 cnt:116647 36065 4674 imps:2.6 lr: 0.0097
Iter:  400/ 9915 loss:0.5105 0.5048 0.5936 0.4717 cnt:112632 39681 4928 imps:2.6 lr: 0.0096
Iter:  450/ 9915 loss:0.5277 0.5012 0.5936 0.5079 cnt:115019 